Estimated runtime: ~2min

# Computing Spatial Gradients
## Comparing Chlorophyll with Sea Surface Temperature

This notebook reads in chloropyll data and computes local gradients in the measured values.
This process is then repeated with sea surface temperature data.

Derivatives are computed using the `FiniteDiff` tool provided in _FiniteDiff.py_, a custom differentiation tool built by Ben Storer.

### To begin, load in our packages of choice

In [ ]:
import numpy as np
import pandas as pd
import xarray as xr

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors as colors

import cmocean
from pyproj import Proj

from AddParallels_and_Meridians import AddParallels_and_Meridians
from FiniteDiff import FiniteDiff

import podaac.podaac as podaac
import podaac.podaac_utils as putil
p = podaac.Podaac()

#### This is only if you use a dark background notebook. Otherwise, comment this out.

In [ ]:
plt.style.use('dark_background')

font = {'size' : 16}
matplotlib.rc('font', **font)

### Define gradient function

In [ ]:
def compute_gradient(field, LAT, ddlat, ddlon):
    grad_field = np.zeros(field.shape)
    
    if (len(field.shape) == 2):
        # Compute (and store) the latitudinal derivative
        dfdlat = np.ma.dot(ddlat, field)
            
        # Compute (and store) the longitudinal derivative
        dfdlon = np.ma.dot(ddlon, field.T).T
            
        # grad(f) =  sec(lat) * ddlon(f) * ehat_lon   +   ddlat(f) * ehat_lat
        # compute (and store) the magnitude of the gradient
        grad_field = np.sqrt(  (dfdlon / np.cos(LAT * np.pi / 180))**2 \
                              + dfdlat**2 )
    else:
        for Itime in range(field.shape[0]):
        
            # Compute (and store) the latitudinal derivative
            dfdlat = np.ma.dot(ddlat, field[Itime,:,:])
            
            # Compute (and store) the longitudinal derivative
            dfdlon = np.ma.dot(ddlon, field[Itime,:,:].T).T
            
            # grad(f) =  sec(lat) * ddlon(f) * ehat_lon   +   ddlat(f) * ehat_lat
            # compute (and store) the magnitude of the gradient
            grad_field[Itime,:,:] = np.sqrt(  (dfdlon / np.cos(LAT * np.pi / 180))**2 \
                                             + dfdlat**2 )
            
    return grad_field

## Specify Data Selection

* `start_date`: datetime object indicating beginning time for selection. In `'YYYY-MM-DD'` format.
* `end_date`: datetime object indicating end time (none-inclusive) for selection. In `'YYYY-MM-DD'` format.
* `VAR`: desired variable. Currently only tested for `'CHL'`
* `ALG`: associated variable algorithm/method. Currently only tested for `'chl_ocx'`
* `BIN`: time-binning period. Currently only accepts `'DAY'` and `'8D'` for dail and 8-day averages, respectively
* `SRES`: spatial resolution. Options are `'4km'` and `'9km'`

In [ ]:
## YYYY-MM-DD
start = '2018-04-12'
end   = '2018-05-12'
#end   = '2018-06-10'

start_date = np.datetime64(start)
end_date   = np.datetime64(end)
num_days = (end_date - start_date).tolist().days

# variable to load
VAR = 'CHL'

# algorithm
ALG = 'chl_ocx'

# Binning period
BIN = '8D'  # DAY, 8D, MO

# Spatial resolution
SRES = '9km'   # 4km, 9km

## Create a list of URLs and associated times

These URLs will then be used to access the requested netcdf datafiles.

In [ ]:
# Build a list of URLs and datetime objects
dap_urls = []
the_days = []

url_base = "https://oceandata.sci.gsfc.nasa.gov:443/opendap/MODISA/L3SMI/"

for ii in range(num_days):
    
    curr_date = start_date + ii
    
    curr_year = curr_date.tolist().year
    ref_date = np.datetime64('{0:d}-01-01'.format(curr_year))
    
    day_num = 1 + (curr_date - ref_date).tolist().days
    
    # We need to change the formatting a bit depending on the binning
    do = True
    if BIN == 'DAY':
        time_str = 'A{0:d}{1:03d}'.format(curr_year, day_num)
    elif BIN == '8D':
        if (day_num - 1) % 8 == 0:
            targ_day = day_num + 7
            if targ_day > 365:
                targ_day = 365
            
            time_str = 'A{0:d}{1:03d}{2:d}{3:03d}'.format(curr_year, day_num, curr_year, targ_day)
        else:
            # There isn't an 8D set starting here
            do = False
    
    if do:
        file_url = url_base + \
                '{0:d}/{1:03d}/{2}'.format(curr_year, day_num, time_str) + \
                '.L3m_{0}_{1}_{2}_{3}'.format(BIN, VAR, ALG, SRES) + \
                '.nc'
    
        dap_urls += [file_url]
        the_days += [curr_date]
    
print('dap_urls containts {0:d} urls for {1} data.'.format(len(dap_urls), VAR))

## Now load the datasets

We don't use `xr.open_mfdataset` because the source datafiles have no time dimension, in addition to having some extraneous variables the cause merging problems.

Instead, we simply create a list of datasets, on for each URL, and in the same order as the URLs.

In [ ]:
data_sets = [xr.open_dataset(url) \
             for (url,ind) \
             in zip(dap_urls, np.arange(num_days))]

### Create the time array corresponding to the datasets

In [ ]:
time_array = xr.DataArray(the_days, None, 'time', 'time')

### Concatenate each separate dataset into one large dataset with a time dimension. 

The values of the time dimension will be taken from `time_array`.

In [ ]:
merged = xr.concat(data_sets, time_array)

## Analysis

We now have the desired dataset 'loaded' into our notebook (recall that it is lazy loading). We can now proceed to analyze the data as we desire!

### Gradients

We're going to compute the magnitude of the spatial gradient ($\left\lvert\\\nabla\mathrm{CHL}\right\lvert$) at each point in space.

To do this, we will need to do a few things in preparation.

1. We'll subset the data to reduce the size of the computations. This isn't strictly necessary, but is useful for illustrative purposes.
2. We'll need to mask out the NaN values. Since differentiation is computed via a matrix multiplication, NaNs will propagate terribly.
3. Compute differentiation matrices for the physical grids (in spherical coordinates)
4. Apply the gradient computation and mask the resulting field using the same mask as applied to the original field.

##### 1. Subsetting

In [ ]:
lon_lb = - 85
lon_ub = - 75

lat_lb =    0
lat_ub =   10

In [ ]:
subs_chl = merged.chl_ocx.sel( lon = slice(lon_lb,lon_ub) ,\
                               lat = slice(lat_ub,lat_lb)  \
                             ).data

subs_lon = merged.lon.sel( lon = slice(lon_lb,lon_ub) ).data
subs_lat = merged.lat.sel( lat = slice(lat_ub,lat_lb) ).data

sLON, sLAT = np.meshgrid(subs_lon, subs_lat)

##### 2. Masking out the NaNs

In [ ]:
subs_chl = np.ma.masked_where(np.isnan(subs_chl), subs_chl)

##### 3. Prepare differentiation matrices

In [ ]:
ddlon_chl = FiniteDiff(subs_lon, 2, uniform=False, spb=False)
ddlat_chl = FiniteDiff(subs_lat, 2, uniform=False, spb=False)

##### 4. Compute Gradient and Apply Previous Mask

In [ ]:
grad_chl      = compute_gradient(subs_chl, sLAT, ddlat_chl, ddlon_chl)
grad_mean_chl = compute_gradient(np.mean(subs_chl, axis=0), sLAT, ddlat_chl, ddlon_chl)

In [ ]:
grad_chl      = np.ma.masked_where(subs_chl.mask, grad_chl)

grad_mean_chl = np.ma.masked_where(np.mean(subs_chl,axis=0).mask, grad_mean_chl)

#### Compare the Different Methods

We computed the mean gradient in two different ways: 
$\overline{\left\lvert\\\nabla\mathrm{CHL}\right\lvert}$ 
and 
$\left\lvert\\\nabla\overline{\mathrm{CHL}}\right\lvert$.

In a perfect world, these would be the same thing. 
However, our data has missing points. 
These points can be missing either because they are over land or because there were clouds, which don't always play well with satellites.

The currently implemented differentiation tools treat masked (missing) points as zero.
Is this a good idea? No, not really. 
But getting to this point was very easy. 
Going past this point will be rather messy.
The takeaway is: *don't trust gradient values near the coast!*

So, why does the order of averaging matter? 
Simply put, if we average first, we can fill in some of the cloudy gaps (clouds move!). 
This avoids the 'setting to zero' issue as much as possible.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12,5), sharey=True)

# Colour map, which sets how to fill masked areas
cmap = plt.get_cmap('cmo.amp')
cmap.set_bad('gray', 1.)

# Plot mean(grad(chl))
to_plot = np.mean(grad_chl, axis=0)
cv = np.percentile(to_plot, 95)
q0 = axes[0].pcolormesh(subs_lon, subs_lat, to_plot, vmin=0, vmax=cv, cmap=cmap)

# Plot grad(mean(chl))
to_plot = grad_mean_chl
cv = np.percentile(to_plot, 95)
q1 = axes[1].pcolormesh(subs_lon, subs_lat, to_plot, vmin=0, vmax=cv, cmap=cmap)

# Add colour bars
plt.colorbar(q0, ax = axes[0])
plt.colorbar(q1, ax = axes[1])

### Now repeat the above process for SST data

We begin by point to an opendap repository and selecting out a data set covering the same time period.

In [ ]:
url = ('https://podaac-opendap.jpl.nasa.gov/opendap/hyrax/'
       'allData/insitu/L2/saildrone/Baja/saildrone-gen_4-baja'
       '_2018-sd1002-20180411T180000-20180611T055959-1_minutes-v1.nc')
ds_usv = xr.open_dataset(url)

In [ ]:
ds_usv2 = ds_usv.isel(trajectory=0).swap_dims({'obs':'time'}).rename({'longitude':'lon','latitude':'lat'})
ds_usv_subset = ds_usv2.sel(time=slice(start+'T02', end+'T18')) 

start_time = pd.to_datetime( str( ds_usv2.time.min().data) ).strftime('%Y-%m-%dT%H:%m:%SZ') 
end_time   = pd.to_datetime( str( ds_usv2.time.max().data) ).strftime('%Y-%m-%dT%H:%m:%SZ') 

print('start: ',start_time,'end: ',end_time)

In [ ]:
#dataset_id = 'PODAAC-GHGMR-4FJ04'  #MUR SST looked up on podaac website
dataset_id = 'PODAAC-GHK10-41N01'  #smaller data
gresult = p.granule_search(dataset_id=dataset_id,
                           start_time=start_time,
                           end_time=end_time,
                           items_per_page='100')
urls = putil.PodaacUtils.mine_opendap_urls_from_granule_search(gresult)
urls = [w[:-5] for w in urls]  #remove html from urls

In [ ]:
ds_sst = xr.open_mfdataset(urls, coords='minimal')

##### 1. Subsetting

In [ ]:
subs_sst = ds_sst.analysed_sst.sel( lon = slice(lon_lb, lon_ub) ,\
                                    lat = slice(lat_ub, lat_lb)  \
                                   )

subs_lon_sst = ds_sst.lon.sel( lon = slice(lon_lb, lon_ub) ).data
subs_lat_sst = ds_sst.lat.sel( lat = slice(lat_ub, lat_lb) ).data

sLON_sst, sLAT_sst = np.meshgrid(subs_lon_sst, subs_lat_sst)

In [ ]:
subs_sst = subs_sst.load()

##### 2. Masking out the NaNs

In [ ]:
subs_sst = np.ma.masked_where(np.isnan(subs_sst), subs_sst)

##### 3. Prepare differentiation matrices

In [ ]:
ddlon_sst = FiniteDiff(subs_lon_sst, 2, uniform=False, spb=False)
ddlat_sst = FiniteDiff(subs_lat_sst, 2, uniform=False, spb=False)

##### 4. Compute Gradient and Apply Previous Mask

In [ ]:
grad_sst = compute_gradient(subs_sst, sLAT_sst, ddlat_sst, ddlon_sst)

grad_mean_sst = compute_gradient(np.mean(subs_sst,axis=0), sLAT_sst, ddlat_sst, ddlon_sst)

In [ ]:
grad_sst      = np.ma.masked_where(subs_sst.mask, grad_sst)

grad_mean_sst = np.ma.masked_where(np.mean(subs_sst,axis=0).mask, grad_mean_sst)

### Compare SST and CHL results

Below, we compare the gradients in sea surface temperature (SST) and chlorophyll (CHL).

In [ ]:
fig, axes = plt.subplots(3, 2, figsize=(12,12), sharey=True, sharex=True)

# Colour map, which sets how to fill masked areas
cmap_c = plt.get_cmap('cmo.algae')
cmap_c.set_bad('gray', 1.)

cmap_s = plt.get_cmap('cmo.amp')
cmap_s.set_bad('gray', 1.)

# Plot mean(chl)
to_plot = np.mean(subs_chl, axis=0)
cv = np.percentile(to_plot, 90)
q00 = axes[0,0].pcolormesh(subs_lon, subs_lat, to_plot, vmin=0, vmax=cv, cmap=cmap_c)

# Plot mean(grad(chl))
to_plot = np.mean(grad_chl, axis=0)
cv = 5#np.percentile(to_plot, 90)
q10 = axes[1,0].pcolormesh(subs_lon, subs_lat, to_plot, vmin=0, vmax=cv, cmap=cmap_c)

# Plot grad(mean(chl))
to_plot = grad_mean_chl
cv = 5#np.percentile(to_plot, 90)
q20 = axes[2,0].pcolormesh(subs_lon, subs_lat, to_plot, vmin=0, vmax=cv, cmap=cmap_c)

# Plot mean(sst)
to_plot = np.mean(subs_sst, axis=0) - 273.15  # Kelvin to Celcius
q01 = axes[0,1].pcolormesh(subs_lon_sst, subs_lat_sst, to_plot, cmap=cmap_s)

# Plot mean(grad(sst))
to_plot = np.mean(grad_sst, axis=0)
cv = 3#np.percentile(to_plot, 90)
q11 = axes[1,1].pcolormesh(subs_lon_sst, subs_lat_sst, to_plot, vmin=0, vmax=cv, cmap=cmap_s)

# Plot grad(mean(sst))
to_plot = grad_mean_sst
#cv = np.percentile(to_plot, 90)
q21 = axes[2,1].pcolormesh(subs_lon_sst, subs_lat_sst, to_plot, vmin=0, vmax=cv, cmap=cmap_s)

# Add colour bars
plt.colorbar(q00, ax = axes[0,0])
plt.colorbar(q01, ax = axes[0,1])
plt.colorbar(q10, ax = axes[1,0])
plt.colorbar(q11, ax = axes[1,1])
plt.colorbar(q20, ax = axes[2,0])
plt.colorbar(q21, ax = axes[2,1])

axes[0,0].set_title('Chlorophyll')
axes[0,1].set_title('SST')

axes[0,0].set_ylabel('Time Mean')
axes[1,0].set_ylabel('Mean of Gradient')
axes[2,0].set_ylabel('Gradient of Mean')